In [114]:
import pandas as pd
import numpy as np

In [115]:
openpath = "E:/Old Downloads Folder/Research Stuff/Climate Data/UHII.csv"

with open(openpath) as f:
    
    data = pd.read_csv(f, sep = ',')

codes = []
    
for index, row in data.iterrows():
    
    code = row['STATION']
    
    codes.append(code)
    
data = data.set_index(['STATION'])

In [116]:
# This code finds the integrals of the first peak from Ovito analyses

s1 = 'E:/Old Downloads Folder/Research Stuff/Jupyter Notebooks/Coordination Analysis Data/'

s2 = '_first_peak.txt'

firstPeakIntegrals = pd.DataFrame()

for code in codes:
    
    openpath = s1 + code + s2
    
    with open(openpath) as f:
    
        val = f.read()

        temp_df = pd.DataFrame([[code, float(val)]], columns = ['Station', 'First Peak Integral'])
        
        firstPeakIntegrals = firstPeakIntegrals.append(temp_df, ignore_index = True)
        
firstPeakIntegrals = firstPeakIntegrals.set_index(['Station'])

In [117]:
# This code finds the data found from QGIS and the average albedo of every area

s1 = 'E:/Old Downloads Folder/Research Stuff/GIS Maps/Albedo Data/'

s2 = '_weighted_albedo.txt'

albedos = pd.DataFrame()

for code in codes:
    
    openpath = s1 + code + s2
    
    with open(openpath) as f:
    
        val = f.read()

        temp_df = pd.DataFrame([[code, float(val)]], columns = ['Station', 'Weighted Albedo'])
        
        albedos = albedos.append(temp_df, ignore_index = True)
        
albedos = albedos.set_index(['Station'])

In [118]:
# This code finds the clustering data from OVITO and finds the location of the first drop

s1 = 'E:/Old Downloads Folder/Research Stuff/Jupyter Notebooks/Coords Folder/Weather Station Coords/First Drops/'

s2 = '_first_drop.txt'

drops = pd.DataFrame()

for code in codes:
    
    openpath = s1 + code + s2
    
    with open(openpath) as f:
    
        val = f.read()

        temp_df = pd.DataFrame([[code, float(val)]], columns = ['Station', 'First Drop'])
        
        drops = drops.append(temp_df, ignore_index = True)
        
drops = drops.set_index(['Station'])

In [119]:
# This code finds the buildings per cluster at the location of the first drop

s1 = 'E:/Old Downloads Folder/Research Stuff/Jupyter Notebooks/Coords Folder/Weather Station Coords/First Drops/Buildings per Cluster/'

s2 = '_buildings_per_cluster.txt'

perCluster = pd.DataFrame()

for code in codes:
    
    openpath = s1 + code + s2
    
    with open(openpath) as f:
    
        val = f.read()

        temp_df = pd.DataFrame([[code, float(val)]], columns = ['Station', 'Buildings Per Cluster'])
        
        perCluster = perCluster.append(temp_df, ignore_index = True)
        
perCluster = perCluster.set_index(['Station'])

In [120]:
# This code finds the data found from QGIS and the average albedo of every area

s1 = 'E:/Old Downloads Folder/Research Stuff/GIS Maps/Albedo Data/'

s2 = '_area_albedo.txt'

areaAlbedos = pd.DataFrame()

for code in codes:
    
    openpath = s1 + code + s2
    
    with open(openpath) as f:
    
        val = f.read()

        temp_df = pd.DataFrame([[code, float(val)]], columns = ['Station', 'Area Albedo'])
        
        areaAlbedos = areaAlbedos.append(temp_df, ignore_index = True)
        
areaAlbedos = areaAlbedos.set_index(['Station'])

In [121]:
# Find the total number of buildings in every station area

import os, os.path

s1 = 'E:/Old Downloads Folder/Research Stuff/GIS Maps/Rooftops/'

buildingsPerStation = pd.DataFrame()

for code in codes:
    
    folder = s1 + code
    
    val = len([name for name in os.listdir(folder) if os.path.isfile(os.path.join(folder, name))])
    
    temp_df = pd.DataFrame([[code, int(val)]], columns = ['Station', 'Buildings In Area'])
        
    buildingsPerStation = buildingsPerStation.append(temp_df, ignore_index = True)

buildingsPerStation = buildingsPerStation.set_index(['Station'])    

In [122]:
# Now we need to merge these frames together

final_frame = pd.DataFrame()

final_frame = pd.concat([data, firstPeakIntegrals, albedos, drops, perCluster, areaAlbedos, buildingsPerStation], axis = 1, sort = False)

final_frame

,UHI,First Peak Integral,Weighted Albedo,First Drop,Buildings Per Cluster,Area Albedo,Buildings In Area
USC00360022,2.411053,2.742999,0.540877,261.630,26.704846,0.43409,24248
USC00360861,5.129269,1.585922,0.535985,421.805,381.944444,0.45598,68750
USC00362574,3.167400,1.396532,0.540247,122.514,6.342829,0.44289,38076
USC00365573,1.490365,1.069074,0.549448,159.995,17.492454,0.44746,44046
USC00365918,2.149054,3.846643,0.538640,137.334,4.805890,0.43075,17950
USW00014762,4.535355,2.152906,0.554050,189.215,25.700230,0.45780,44667
USW00094823,3.455903,2.562728,0.554652,137.287,4.454569,0.43474,16865


In [123]:
# Now to do the regression analysis

from sklearn.linear_model import LinearRegression

# Create linear regression object

mlr = LinearRegression()

# Fit linear regression

mlr.fit(final_frame[['First Peak Integral', 'Weighted Albedo', 'First Drop', 'Buildings Per Cluster', 'Area Albedo', 'Buildings In Area']], final_frame['UHI'])

# Get the slope and intercept of the line best fit

print(mlr.intercept_)

print(mlr.coef_)

-112.18734002181793
[-1.64828863e+00 -2.30252349e+02 -3.90461783e-03  2.62007201e-02
  5.89540757e+02 -4.92168318e-04]


In [124]:
from sklearn.metrics import r2_score

y_true = final_frame['UHI'].tolist()

y_pred = []

for index, data in final_frame.iterrows():
    
    pred = mlr.intercept_ + data['First Peak Integral'] * mlr.coef_[0] + data['Weighted Albedo'] * mlr.coef_[1] + data['First Drop'] * mlr.coef_[2] + data['Buildings Per Cluster'] * mlr.coef_[3] + data['Area Albedo'] * mlr.coef_[4] + data['Buildings In Area'] * mlr.coef_[5]
    
    y_pred.append(pred)

print('The R2 value using all the predictor variables is: ' + str(r2_score(y_true, y_pred)))

The R2 value using all the predictor variables is: 1.0


In [125]:
plot_frame = final_frame.copy()

plot_frame['Predicted UHI'] = y_pred

plot_frame['UHI Diff'] = plot_frame['Predicted UHI'] - plot_frame['UHI']

plot_frame

,UHI,First Peak Integral,Weighted Albedo,First Drop,Buildings Per Cluster,Area Albedo,Buildings In Area,Predicted UHI,UHI Diff
USC00360022,2.411053,2.742999,0.540877,261.630,26.704846,0.43409,24248,2.411053,4.085940e-10
USC00360861,5.129269,1.585922,0.535985,421.805,381.944444,0.45598,68750,5.129269,-4.225686e-11
USC00362574,3.167400,1.396532,0.540247,122.514,6.342829,0.44289,38076,3.167400,-6.747527e-10
USC00365573,1.490365,1.069074,0.549448,159.995,17.492454,0.44746,44046,1.490365,-6.390053e-10
USC00365918,2.149054,3.846643,0.538640,137.334,4.805890,0.43075,17950,2.149054,1.002473e-09
USW00014762,4.535355,2.152906,0.554050,189.215,25.700230,0.45780,44667,4.535355,-1.465423e-10
USW00094823,3.455903,2.562728,0.554652,137.287,4.454569,0.43474,16865,3.455903,9.136514e-11


In [126]:
# This bit of code is my attempt to find the most important predictor variables

predict_frame = final_frame.copy().drop(['UHI'], axis = 1)

numCols = len(predict_frame.columns)

y_true = final_frame['UHI'].tolist()

for i in range(numCols):
    
    predictorCols = np.arange(1, numCols + 1)

    predictorCols = np.delete(predictorCols, i)
    
    colNames = []
    
    for col in predictorCols:
        
        colNames.append(predict_frame.columns[col - 1])
        
    mlr.fit(final_frame[colNames], final_frame['UHI'])
    
    y_pred = []
    
    for index, data in final_frame.iterrows():
    
        pred = mlr.intercept_
        
        for item in colNames:
            
            pred += data[item] * mlr.coef_[colNames.index(item)]
    
        y_pred.append(pred)
    
    print('The R2 value is: ' + str(round(r2_score(y_true, y_pred), 2)), '--', \
          'The dropped variable is: ' + str(predict_frame.columns[i]))

The R2 value is: 0.86 -- The dropped variable is: First Peak Integral
The R2 value is: 0.82 -- The dropped variable is: Weighted Albedo
The R2 value is: 0.99 -- The dropped variable is: First Drop
The R2 value is: 0.63 -- The dropped variable is: Buildings Per Cluster
The R2 value is: 0.63 -- The dropped variable is: Area Albedo
The R2 value is: 0.69 -- The dropped variable is: Buildings In Area
